# Create MISP objects with PyMISP

This notebook demonstrates how to create MISP objects, create relations between objects and how to get details on the MISP object definitions.

In [ ]:
import urllib3
import sys
import json
from pyfaup.faup import Faup
from pymisp import *

sys.path.insert(0, "../vault/")
from keys import *
if misp_verifycert is False:
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
print("\033[92mPython libraries\033[90m are loaded and the \033[92mcredentials\033[90m are read from keys file.")

# PyMISP object and header dictionary for requests outside PyMISP
misp = PyMISP(misp_url, misp_key, misp_verifycert)
print("I will use the MISP server \033[92m{}\033[90m for this playbook.".format(misp_url))

## MISP objects

### What are MISP objects?

MISP objects are **containers around contextually linked attributes**. They support analysts in grouping related attributes and describing the relations that exist between the data points in a threat event. Combining these objects and relations is something that can then be used to represent the story of what is being told in the threat event.

MISP supports a large list of objects such as

- AV-signature: an antivirus detection signature
- Domain/IP: document a tuple domain, hostname and IP address
- File: describing a file with meta information
- Organization: an object to describe an organization

All these objects are available via the [MISP-Objects Github repository](https://github.com/MISP/misp-objects) and directly accessible within MISP.

### Adding MISP objects to an event

You can add MISP objects to an event via **Add object** in the left menu bar. This is useful for manually adding objects but  the real power comes from automating the process and adding objects to MISP via PyMISP.

### MISP object definitions

Before you add an object to MISP it is useful to know the requirements, or the attributes, that are essential for an object. You can do this via the GitHub repository but you can also do this directly with PyMISP. The function `get_raw_object_template` queries MISP for the raw template in your MISP system.

In [ ]:
for template in ["domain-ip", "url"]:
    print("Object template definition for {}\n------------------------------------\n".format(template))
    object_definition = misp.get_raw_object_template(template)
    print(json.dumps(object_definition, indent=2))

### Create a MISP object with PyMISP

Now that we know the object definition, it's time to create two objects. In this example we create two MISP objects. A `url` object, representing a web url and a `domain-ip` object. We have learned from the previous section that the URL object requires a url (or a resource_path) and that domain-ip requires one of ip, domain or hostname.

We use the URL `https://www.misp-project.org/` and will use the Faup library to extract the domain of the URL.

In [ ]:
url = "https://www.misp-project.org/"
misp_event_uuid = "5bee196d-1ed5-418a-b032-32760b477c88"

In [ ]:
fp = Faup()
fp.decode(url)
phishing_url_parsed = fp.get()
domain = phishing_url_parsed.get("domain", False)
if domain:
    # Create the MISP URL object
    url_object = MISPObject("url")
    url_object.add_attribute("url", url)
    url_object.add_attribute("domain", domain)    
    url_object.add_attribute("host", phishing_url_parsed.get("host", False))
    url_object.add_attribute("scheme", phishing_url_parsed.get("scheme", False))

    # Create the MISP domain-ip object
    domain_ip_object = MISPObject("domain-ip")
    domain_ip_object.add_attribute("domain", domain)
    
    # Add the objects to MISP
    url_object_misp = misp.add_object(misp_event_uuid, url_object, pythonify=True)
    print("Added URL object \033[92m{}\033[90m".format(url_object_misp.uuid))
    domain_ip_object_misp = misp.add_object(misp_event_uuid, domain_ip_object, pythonify=True)   
    print("Added domain-ip object \033[92m{}\033[90m".format(url_object_misp.uuid))
    
    # Add a relationship between the objects
    reference = misp.add_object_reference(url_object_misp.add_reference(domain_ip_object_misp.uuid, "related-to"), pythonify=True)
    print("Added relation between URL and domain-ip object \033[92m{}\033[90m".format(reference.uuid))